In [1]:
import matplotlib
matplotlib.use("Agg")
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.models import load_model
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import backend as K
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import pickle
import cv2
import os
from keras.callbacks import EarlyStopping
from skimage import exposure

In [2]:
data = []
labels = []
image_data_folder_path = "C:/Users/surya/OneDrive/Desktop/Alzheimers/Alzheimer_s Dataset"
imagePaths = sorted(list(paths.list_images(image_data_folder_path)))
total_number_of_images = len(imagePaths)
print("Total number of images : ",total_number_of_images)
random.shuffle(imagePaths)

Total number of images :  6400


In [3]:
for imagePath in imagePaths:
    image = cv2.imread(imagePath,cv2.IMREAD_GRAYSCALE)
    image1 = exposure.equalize_adapthist(image, clip_limit=0.03)
    data.append(image1)
    label = imagePath.split(os.path.sep)[-2]
    labels.append(label)
print ("data",data[0].shape)

data (208, 176)


In [4]:
# scale the raw pixel intensities to the range [0, 1]
#data = np.array(data, dtype="float") / 255.0
data = np.array(data, dtype="float")
labels = np.array(labels)

In [5]:
(X_train,X_test,y_train,y_test) = train_test_split(data,labels, test_size=0.2, stratify = labels,shuffle = True, random_state=100)
print ("X_train shape:",X_train.shape)
print ("X_test shape:",X_test.shape)

X_train shape: (5120, 208, 176)
X_test shape: (1280, 208, 176)


In [6]:
lb = LabelBinarizer()
y_train = lb.fit_transform(y_train)
y_test= lb.transform(y_test)

In [7]:
print('length X_train:', len(X_train))
print('length X_test:',  len(X_test))
print('length y_train:', len(y_train))
print('length y_test:', len(y_test))

length X_train: 5120
length X_test: 1280
length y_train: 5120
length y_test: 1280


In [8]:
X_train = np.array(X_train).reshape(5120,208,176,1)
X_test = np.array(X_test).reshape(1280,208,176,1)

In [9]:
height = 208
width = 176
depth=1

inputShape = (height, width,depth)

classes = len(lb.classes_)

In [10]:
lb.classes_

array(['MildDemented', 'ModerateDemented', 'NonDemented',
       'VeryMildDemented'], dtype='<U16')

In [11]:
from sklearn.utils import compute_class_weight

y_integers = np.argmax(y_train, axis=1)
class_weights = compute_class_weight('balanced', np.unique(y_integers), y_integers)
class_weights1 = dict(enumerate(class_weights))
print(class_weights1)

{0: 1.7852161785216178, 1: 25.098039215686274, 2: 0.5, 3: 0.7142857142857143}


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass classes=[0 1 2 3], y=[3 2 2 ... 2 2 2] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


# CNN

In [12]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization

model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(208,176,1)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))


model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax')) 

model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 206, 174, 32)      320       
_________________________________________________________________
batch_normalization (BatchNo (None, 206, 174, 32)      128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 103, 87, 32)       0         
_________________________________________________________________
dropout (Dropout)            (None, 103, 87, 32)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 101, 85, 64)       18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 101, 85, 64)       256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 50, 42, 64)        0

In [14]:
INIT_LR = 1e-3
EPOCHS = 100
BS = 32
NUM_CLASSES = 4


In [15]:
from keras.callbacks import EarlyStopping,ModelCheckpoint, ReduceLROnPlateau

checkpoint_cb = ModelCheckpoint("AD_Stages_model.h5", save_best_only=True)

learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=10, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.001)

callbacks=[checkpoint_cb, learning_rate_reduction]


In [16]:
history = model.fit(X_train,y_train, batch_size=BS,validation_data=(X_test,y_test), steps_per_epoch=len(X_train) // BS,epochs=EPOCHS, 
                        class_weight = class_weights1,callbacks=callbacks)

Epoch 1/100
160/160 [==============================] - 377s 2s/step - loss: 1.5126 - accuracy: 0.4159 - val_loss: 82.4167 - val_accuracy: 0.1398
Epoch 2/100
160/160 [==============================] - 405s 3s/step - loss: 0.7015 - accuracy: 0.6112 - val_loss: 31.8369 - val_accuracy: 0.1523
Epoch 3/100
160/160 [==============================] - 373s 2s/step - loss: 0.4988 - accuracy: 0.7167 - val_loss: 220.8446 - val_accuracy: 0.0102
Epoch 4/100
160/160 [==============================] - 353s 2s/step - loss: 0.3517 - accuracy: 0.7932 - val_loss: 15.3746 - val_accuracy: 0.0836
Epoch 5/100
160/160 [==============================] - 348s 2s/step - loss: 0.2420 - accuracy: 0.8687 - val_loss: 1.1122 - val_accuracy: 0.6023
Epoch 6/100
160/160 [==============================] - 348s 2s/step - loss: 0.1977 - accuracy: 0.8931 - val_loss: 0.3402 - val_accuracy: 0.8711
Epoch 7/100
160/160 [==============================] - 348s 2s/step - loss: 0.1662 - accuracy: 0.9034 - val_loss: 0.9640 - val_accu

In [17]:
model.save_weights('my_model_filter.h5')

In [18]:
print("[INFO] evaluating network...")
predictions = model.predict(X_test, batch_size=10)
print(classification_report(y_test.argmax(axis=1),predictions.argmax(axis=1), target_names=lb.classes_))

[INFO] evaluating network...
                  precision    recall  f1-score   support

    MildDemented       0.99      0.98      0.99       179
ModerateDemented       1.00      1.00      1.00        13
     NonDemented       0.97      1.00      0.98       640
VeryMildDemented       1.00      0.96      0.98       448

        accuracy                           0.98      1280
       macro avg       0.99      0.98      0.99      1280
    weighted avg       0.98      0.98      0.98      1280



In [30]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
pred_ls1 = np.argmax(predictions,axis=1)
test_ls1 = np.argmax(y_test,axis=1)
conf_arr1 = confusion_matrix(test_ls1, pred_ls1)
conf_arr1

array([[175,   0,   4,   0],
       [  0,  13,   0,   0],
       [  0,   0, 640,   0],
       [  1,   0,  16, 431]], dtype=int64)

In [71]:
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
conf_arr1 = confusion_matrix(test_ls1, pred_ls1)
sns.heatmap(conf_arr1, cmap='Blues', annot=True) 
plt.imshow(conf_arr1, cmap=plt.cm.Blues)
plt.xlabel("Predicted labels")
plt.ylabel("True labels")
plt.title('Confusion matrix ')
plt.colorbar()
plt.show()

In [42]:
epoches = np.arange(1,len(history.history.get('loss'))+1)
plt.plot(epoches, history.history.get('loss'), 'b',label='Loss')
plt.plot(epoches, history.history.get('val_loss'),'k', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [45]:
plt.plot(epoches, history.history.get('accuracy'), 'b',label='Accuracy')
plt.plot(epoches, history.history.get('val_accuracy'),'k', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.show()